## **Sum Of Squares** 
Notebook to test the algorithm for representing polynomials as sums of squares.

In [33]:
import itertools, time, cvxpy as cp, numpy as np, sympy as sp, symengine as se
from sympy import symbols, expand
x1, x2, x3, x4, x5, x6, x7 = symbols('x1 x2 x3 x4 x5 x6 x7')

In [34]:
def sdp(matrix, threshold=1e-4): 
    if matrix.shape[0] != matrix.shape[1]:
        return False
    
    eigenvalues, _ = np.linalg.eig(matrix)
    eigenvalues[np.abs(eigenvalues) < threshold] = 0

    return np.all(eigenvalues >= 0)

In [35]:
def decomposition(matrix):
    no_rep_monom = set()
    desc_matrices = list()

    for row in matrix:
        for element in row:
            no_rep_monom.add(tuple(element))

    for mon in no_rep_monom:
        desc_matrix = []
        for row in matrix:
            new_row = []
            for element in row:
                if tuple(element) == mon:
                    new_row.append(1)
                else:
                    new_row.append(0)
            desc_matrix.append(new_row)

        desc_matrices.append(desc_matrix)

    return list(no_rep_monom), desc_matrices

In [36]:
def expand_expression(matrix, vd):
    matrix = np.array(matrix)
    
    num_vars = len(vd[0])
    variables = sp.symbols(f'x1:{num_vars+1}')
    
    variable_terms = []
    for exponents in vd:
        if all(e == 0 for e in exponents):
            term = 1  # (0,0) is 1
        else:
            term = sp.Mul(*(variables[j]**exponents[j] for j in range(num_vars) if exponents[j] != 0))
        variable_terms.append(term)
    
    expression = 0
    for col in range(matrix.shape[1]):
        term = 0
        for row in range(matrix.shape[0]):
            term += matrix[row, col] * variable_terms[row]  
        expression += term**2  

    return se.expand(expression)

In [37]:
def prod_decomposition(matrix):
    U, s, Vt = np.linalg.svd(matrix) 
    s_sqrt = np.sqrt(s)
    S_sqrt = np.diag(s_sqrt)
    
    H = np.dot(U, S_sqrt)

    """
    # Check that Q = H * H^T
    reconstructed_Q = np.dot(H, H.T)
    
    print("\nReconstructed Q:")
    print(reconstructed_Q)
    print("\nH:")
    """
    
    return H

In [38]:
def check_output(polynomial, solver):
    if solver == 'SCS':
        tol = 0.00001
        
    elif solver == 'MOSEK':
        tol = 0.00000001
            
    f_monomials = [term[1] for term in f]
    out_result = 1
    for term in f:
        found = False
        for element in polynomial:
            if element[1] == term[1]:
                found = True
                if abs(element[0]-term[0]) >= tol:
                    out_result = 0
                break
        
        if out_result == 0:
            break

        if found == False:
            print(term)
            out_result = 0
            break
    
    for element in polynomial:
        if element[1] not in f_monomials and abs(element[0]) >= tol: 
            out_result = 0
            break

    return out_result

In [39]:
def generate_monomials(n_variables, deg):
    monomials = []
    for comb in itertools.product(range(deg + 1), repeat=n_variables):
        if sum(comb) <= deg and comb not in monomials:
            if n_variables == 1:
                monomials.append((comb[0], 0))
            else:
                monomials.append(comb)
    return monomials

In [49]:
"""
We can choose the solver: SCS or MOSEK.
"""
def SOS(f, vd, solver):
    Q = cp.Variable((len(vd),len(vd)), symmetric=True)
    constraints = [Q >> 0]

    # Matrix of monomials resulting from vd * vd'
    prod_vd = [[tuple(a[i] + c[i] for i in range(len(vd[0]))) for c in vd] for a in vd] 
    
    # Decompose into the matrices B_alpha
    l1, l2 = decomposition(prod_vd)
    
    # Set tr(Q, B_alpha) = g_alpha
    for monomial in l1:
        is_in = False
        for coef, monom in f:
            if monomial == monom:
                constraints += [cp.trace(Q @ l2[l1.index(monomial)]) == coef]
                is_in = True
                break
                
        if is_in == False:
            constraints += [cp.trace(Q @ l2[l1.index(monomial)]) == 0]

    prob = cp.Problem(cp.Minimize(0), constraints)
    
    try:
        if solver == 'SCS':
            prob.solve(solver='SCS', verbose=False)
            
        elif solver == 'MOSEK':
            prob.solve(solver='MOSEK', verbose=False, mosek_params = {'MSK_IPAR_NUM_THREADS':  3})
        
        if Q.value is not None:
            pass
            """
            print(sdp(Q.value))
            print("....................................................................")
            print(Q.value)
            """
        else:
            print("No sum of squares decomposition found.")
        return Q.value
            
    except cp.error.SolverError as e:
        print("Error with the solver:", e)
        return None

In [50]:
"""
Method responsible for the general logic.
"""
def main(f, solver):
    univariate = all(len(monomial) == 2 and monomial[1] == 0 for _, monomial in f)
    if univariate:
        num_vars = 1
    else:
        num_vars = len(f[0][1])
    deg_f = max(sum(monomial) for _, monomial in f)
    vd = generate_monomials(num_vars, deg_f // 2) # It could be considered to handle homogeneous cases to reduce the size of vd

    Q = SOS(f, vd, solver)

    # We check if we have obtained a solution and, if so, whether it is sufficiently accurate
    if Q is not None:
        H = prod_decomposition(Q)
        expr = expand_expression(np.matrix(H), vd)

        expr = sp.expand(expr)
        terms = expr.as_ordered_terms()
        variables = expr.free_symbols
        variables = sorted(variables, key=lambda x: str(x))  

        result = []
        for term in terms:
            coef, mon = term.as_coeff_Mul()  
            if mon == 1:  
                tuple_mon = (0,) * len(variables)
            else:
                tuple_mon = tuple(mon.as_powers_dict().get(var, 0) for var in variables)

            if univariate: # If we are in the univariate case, we need to properly format the polynomial structure
                tuple_mon = (tuple_mon[0], 0)
            result.append((coef, tuple_mon))

        if len(result[0][1]) == 0:
            out_result = -1  # No solution found
        else:
            out_result = check_output(result, solver)  # 0: solution not accepted as valid, 1: accepted

        print(out_result)
        print("------------------------")
        print(expr)

#### **Example 1**
$f(x_1) = (x_1+2)^{10}$

In [194]:
sp.expand(((x1+2)**10))

x1**10 + 20*x1**9 + 180*x1**8 + 960*x1**7 + 3360*x1**6 + 8064*x1**5 + 13440*x1**4 + 15360*x1**3 + 11520*x1**2 + 5120*x1 + 1024

In [51]:
# ---------------- User input ----------------
f = [(1,(10, 0)), (20,(9, 0)), (180,(8, 0)), (960,(7, 0)), (3360,(6, 0)), (8064,(5, 0)), (13440,(4, 0)),(15360,(3, 0)), (11520,(2, 0)),
     (5120,(1, 0)), (1024,(0, 0))]
# -------------------------------------------------
main(f, 'SCS')

0
------------------------
2.40240402848373*x1**10 + 18.1282099897303*x1**9 + 181.844931208088*x1**8 + 958.181408891589*x1**7 + 3361.85113459525*x1**6 + 8062.11785096697*x1**5 + 13441.1338598464*x1**4 + 15359.3111921711*x1**3 + 11520.4452682507*x1**2 + 5119.73126875358*x1 + 1024.12139260313


In [52]:
# ---------------- User input ----------------
f = [(1,(10, 0)), (20,(9, 0)), (180,(8, 0)), (960,(7, 0)), (3360,(6, 0)), (8064,(5, 0)), (13440,(4, 0)),(15360,(3, 0)), (11520,(2, 0)),
     (5120,(1, 0)), (1024,(0, 0))]
# -------------------------------------------------
main(f, 'MOSEK')

Error with the solver: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.


#### **Example 2**
$f(x_1) = (x_1+2)^8$

In [198]:
sp.expand(((x1+2)**8))

x1**8 + 16*x1**7 + 112*x1**6 + 448*x1**5 + 1120*x1**4 + 1792*x1**3 + 1792*x1**2 + 1024*x1 + 256

In [23]:
# ---------------- User input ----------------
f = [(1,(8, 0)), (16,(7, 0)), (112,(6, 0)), (448,(5, 0)), (1120,(4, 0)),(1792,(3, 0)), (1792,(2, 0)), (1024,(1, 0)), (256,(0, 0))]
# -------------------------------------------------
main(f, 'SCS')

0
------------------------
1.05899261774886*x1**8 + 15.9445650505186*x1**7 + 112.048217338416*x1**6 + 447.957049199821*x1**5 + 1120.03907775391*x1**4 + 1791.98230370953*x1**3 + 1792.00890768161*x1**2 + 1023.99564556658*x1 + 256.001689651118


In [231]:
# ---------------- User input ----------------
f = [(1,(8, 0)), (16,(7, 0)), (112,(6, 0)), (448,(5, 0)), (1120,(4, 0)),(1792,(3, 0)), (1792,(2, 0)), (1024,(1, 0)), (256,(0, 0))]
# -------------------------------------------------
main(f, 'MOSEK')

0
------------------------
0.999999999946885*x1**8 + 15.9999999945626*x1**7 + 111.999999963241*x1**6 + 447.99999985251*x1**5 + 1119.99999963145*x1**4 + 1791.99999941026*x1**3 + 1791.99999941027*x1**2 + 1023.99999966301*x1 + 255.999999915752


#### **Example 3**
$f(x_1) = (x_1+2)^6$

In [247]:
sp.expand(((x1+2)**6))

x1**6 + 12*x1**5 + 60*x1**4 + 160*x1**3 + 240*x1**2 + 192*x1 + 64

In [248]:
# ---------------- User input ----------------
f = [(1,(6, 0)), (12,(5, 0)), (60,(4, 0)),(160,(3, 0)), (240,(2, 0)), (192,(1, 0)), (64,(0, 0))]
# -------------------------------------------------
main(f, 'SCS')

0
------------------------
1.0021468313736*x1**6 + 11.9978188676302*x1**5 + 60.0016834467235*x1**4 + 159.998812205006*x1**3 + 240.000425927072*x1**2 + 191.99986907195*x1 + 64.000029601721


In [249]:
# ---------------- User input ----------------
f = [(1,(6, 0)), (12,(5, 0)), (60,(4, 0)),(160,(3, 0)), (240,(2, 0)), (192,(1, 0)), (64,(0, 0))]
# -------------------------------------------------
main(f, 'MOSEK')

0
------------------------
1.00000000003208*x1**6 + 11.999999999218*x1**5 + 59.9999999965004*x1**4 + 159.999999990505*x1**3 + 239.999999985817*x1**2 + 191.999999988639*x1 + 63.9999999962159


#### **Example 4**
$f(x_1) = (x_1+2)^4$

In [250]:
sp.expand(((x1+2)**4))

x1**4 + 8*x1**3 + 24*x1**2 + 32*x1 + 16

In [251]:
# ---------------- User input ----------------
f = [(1,(4, 0)),(8,(3, 0)), (24,(2, 0)), (32,(1, 0)), (16,(0, 0))]
# -------------------------------------------------
main(f, 'SCS')

1
------------------------
1.00000026603636*x1**4 + 7.99999976711743*x1**3 + 24.0000001533744*x1**2 + 31.9999999600795*x1 + 16.0000000074669


In [252]:
# ---------------- User input ----------------
f = [(1,(4, 0)),(8,(3, 0)), (24,(2, 0)), (32,(1, 0)), (16,(0, 0))]
# -------------------------------------------------
main(f, 'MOSEK')

0
------------------------
1.00000000024567*x1**4 + 7.99999999551832*x1**3 + 23.9999999886854*x1**2 + 31.9999999842426*x1 + 15.9999999922395


#### **Example 5**
$f(x_1, x_2) = 2x_1^4+2x_1^3x_2-x_1^2x_2^2+5x_2^4-1$

In [43]:
# ---------------- User input ----------------
f = [(2,(4, 0)), (2,(3, 1)), (-1,(2, 2)), (5,(0, 4)),(-1,(0,0))] 
# -------------------------------------------------
main(f, 'SCS')

No sum of squares decomposition found.


In [44]:
# ---------------- User input ----------------
f = [(2,(4, 0)), (2,(3, 1)), (-1,(2, 2)), (5,(0, 4)),(-1,(0,0))] 
# -------------------------------------------------
main(f, 'MOSEK')

No sum of squares decomposition found.


#### **Example 6**
$f(x_1, x_2) = 2x_1^4+2x_1^3x_2-x_1^2x_2^2+5x_2^4$

In [234]:
# ---------------- User input ----------------
f = [(2,(4, 0)), (2,(3, 1)), (-1,(2, 2)), (5,(0, 4))] 
# -------------------------------------------------
main(f, 'SCS')

1
------------------------
1.99999999784159*x1**4 + 1.99999998884981*x1**3*x2 + 1.18956203413041e-17*x1**3 - 0.999999930424641*x1**2*x2**2 + 2.9232227738859e-16*x1**2*x2 - 5.21895179628575e-16*x1**2 + 5.29683167327377e-8*x1*x2**3 + 5.1569279455641e-17*x1*x2**2 - 9.23754152995882e-18*x1*x2 - 1.00056962135092e-19*x1 + 5.00000008422844*x2**4 + 1.4241682769556e-15*x2**3 + 1.34734288647454e-15*x2**2 + 5.48190910771327e-16*x2 + 2.03876979361102e-17


In [235]:
# ---------------- User input ----------------
f = [(2,(4, 0)), (2,(3, 1)), (-1,(2, 2)), (5,(0, 4))] 
# -------------------------------------------------
main(f, 'MOSEK')

0
------------------------
1.99999998188141*x1**4 + 1.99999998189944*x1**3*x2 + 5.99382192905956e-20*x1**3 - 0.999999990954786*x1**2*x2**2 - 2.67879101908206e-17*x1**2*x2 - 4.98223736030388e-13*x1**2 + 3.4928726577732e-12*x1*x2**3 + 4.49156548691432e-17*x1*x2**2 - 1.50766910823777e-12*x1*x2 - 7.81600603099801e-20*x1 + 4.99999995470802*x2**4 - 8.29193568337067e-18*x2**3 - 4.02626399559868e-12*x2**2 + 7.23657159998269e-16*x2 + 2.05838510916413e-8


#### **Example 7**
$f(x_1) = x_1^4+x_1^3+x_1^2+x_1-1$

In [45]:
# ---------------- User input ----------------
f = [(-1,(0, 0)), (1,(1, 0)), (1,(2, 0)), (1,(3, 0)), (1,(4, 0))] 
# -------------------------------------------------
main(f, 'SCS')

No sum of squares decomposition found.


In [46]:
# ---------------- User input ----------------
f = [(-1,(0, 0)), (1,(1, 0)), (1,(2, 0)), (1,(3, 0)), (1,(4, 0))] 
# -------------------------------------------------
main(f, 'MOSEK')

No sum of squares decomposition found.


#### **Example 8**
$f(x_1) = x_1^4+x_1^3+x_1^2+x_1+1$

In [238]:
# ---------------- User input ----------------
f = [(1,(0, 0)), (1,(1, 0)), (1,(2, 0)), (1,(3, 0)), (1,(4, 0))] 
# -------------------------------------------------
main(f, 'SCS')

1
------------------------
1.0000000197372*x1**4 + 1.00000001900369*x1**3 + 1.00000001710437*x1**2 + 1.00000001900369*x1 + 1.0000000197372


In [239]:
# ---------------- User input ----------------
f = [(1,(0, 0)), (1,(1, 0)), (1,(2, 0)), (1,(3, 0)), (1,(4, 0))] 
# -------------------------------------------------
main(f, 'MOSEK')

1
------------------------
0.999999999309213*x1**4 + 0.999999999606528*x1**3 + 0.99999999940401*x1**2 + 0.999999999606528*x1 + 0.999999999309213


#### **Example 9**
$f(x_1, x_2) = (x_1+2)^{14}+x_2^2$

In [225]:
sp.expand(((x1+2)**14) + x2**2)

x1**14 + 28*x1**13 + 364*x1**12 + 2912*x1**11 + 16016*x1**10 + 64064*x1**9 + 192192*x1**8 + 439296*x1**7 + 768768*x1**6 + 1025024*x1**5 + 1025024*x1**4 + 745472*x1**3 + 372736*x1**2 + 114688*x1 + x2**2 + 16384

In [255]:
# ---------------- User input ----------------
f = [(1,(14, 0)), (28,(13, 0)), (364,(12, 0)), (2912,(11, 0)), (16016,(10, 0)), (64064,(9, 0)), (192192,(8, 0)), (439296,(7, 0)), (768768,(6, 0)), 
     (1025024,(5, 0)), (1025024,(4, 0)), (745472,(3, 0)), (372736,(2, 0)), (114688,(1, 0)), (1,(0, 2)), (16384,(0, 0))] # -------------------------------------------------
# -------------------------------------------------
main(f, 'SCS')

0
------------------------
14.6114167291473*x1**14 + 0.00198352676730973*x1**13*x2 + 13.3131522358223*x1**13 - 0.0101417159943669*x1**12*x2**2 + 0.0417717187650657*x1**12*x2 + 368.797726978962*x1**12 + 0.00292862691424025*x1**11*x2**3 - 0.0641666665895725*x1**11*x2**2 + 0.0838970307647928*x1**11*x2 + 2880.09796156319*x1**11 + 0.0127855703942715*x1**10*x2**4 + 0.00620201556126161*x1**10*x2**3 + 0.080799868938983*x1**10*x2**2 - 0.130682595353795*x1**10*x2 + 16077.0293821978*x1**10 + 0.000706337387994573*x1**9*x2**5 + 0.000693026463085966*x1**9*x2**4 - 0.00314666094328151*x1**9*x2**3 - 0.133776836603935*x1**9*x2**2 - 0.0519629325249703*x1**9*x2 + 63966.7152166087*x1**9 - 0.0018668044056357*x1**8*x2**6 + 0.00257229607896917*x1**8*x2**5 - 0.0164382930073461*x1**8*x2**4 - 0.00112628029179066*x1**8*x2**3 + 0.217137342506155*x1**8*x2**2 + 0.232181801707719*x1**8*x2 + 192331.231204547*x1**8 - 0.000210768244344371*x1**7*x2**7 - 0.00104551255452443*x1**7*x2**6 + 0.00069539670042245*x1**7*x2**5 + 

In [47]:
# ---------------- User input ----------------
f = [(1,(14, 0)), (28,(13, 0)), (364,(12, 0)), (2912,(11, 0)), (16016,(10, 0)), (64064,(9, 0)), (192192,(8, 0)), (439296,(7, 0)), (768768,(6, 0)), 
     (1025024,(5, 0)), (1025024,(4, 0)), (745472,(3, 0)), (372736,(2, 0)), (114688,(1, 0)), (1,(0, 2)), (16384,(0, 0))] # -------------------------------------------------
# -------------------------------------------------
main(f, 'MOSEK')

Error with the solver: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.


#### **Example 10**
$f(x_1, x_2) = (x_1+2)^8+x_2^2$

In [243]:
sp.expand(((x1+2)**8) + x2**2)

x1**8 + 16*x1**7 + 112*x1**6 + 448*x1**5 + 1120*x1**4 + 1792*x1**3 + 1792*x1**2 + 1024*x1 + x2**2 + 256

In [257]:
# ---------------- User input ----------------
f = [(1,(8, 0)), (16,(7, 0)), (112,(6, 0)), (448,(5, 0)), (1120,(4, 0)),(1792,(3, 0)), (1792,(2, 0)), (1024,(1, 0)), (1,(0, 2)), (256,(0, 0))] 
# -------------------------------------------------
main(f, 'SCS')

0
------------------------
1.058614565167*x1**8 - 3.4807972784293e-9*x1**7*x2 + 15.9441236270089*x1**7 - 0.00086968819582545*x1**6*x2**2 - 9.86880299276804e-9*x1**6*x2 + 112.048740761765*x1**6 + 1.23730058460002e-10*x1**5*x2**3 - 0.00428522352907345*x1**5*x2**2 + 1.21112837480556e-8*x1**5*x2 + 447.956737878492*x1**5 + 6.73652489361515e-5*x1**4*x2**4 + 9.11909688835212e-10*x1**4*x2**3 - 0.0099517145206279*x1**4*x2**2 - 5.26618768258653e-9*x1**4*x2 + 1120.039148965*x1**4 - 3.78916037545337e-12*x1**3*x2**5 + 0.000262681788886262*x1**3*x2**4 + 1.76999302193708e-9*x1**3*x2**3 + 0.00406177287696991*x1**3*x2**2 + 3.75226780555255e-9*x1**3*x2 + 1791.98216095165*x1**3 + 3.59550569311055e-6*x1**2*x2**6 + 3.38824079088726e-12*x1**2*x2**5 + 0.000545903489708819*x1**2*x2**4 + 9.77508876939913e-10*x1**2*x2**3 - 0.0023908198393655*x1**2*x2**2 - 3.46776103443406e-9*x1**2*x2 + 1792.0089399823*x1**2 - 5.79484723634805e-14*x1*x2**7 + 7.56214004902332e-6*x1*x2**6 + 1.64296699991332e-11*x1*x2**5 + 0.000663

In [48]:
# ---------------- User input ----------------
f = [(1,(8, 0)), (16,(7, 0)), (112,(6, 0)), (448,(5, 0)), (1120,(4, 0)),(1792,(3, 0)), (1792,(2, 0)), (1024,(1, 0)), (1,(0, 2)), (256,(0, 0))] 
# -------------------------------------------------
main(f, 'MOSEK')

Error with the solver: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.


#### **Example 11**
$f(x_1) = x_1^{12}+x_1^8$

In [259]:
# ---------------- User input ----------------
f = [(1,(12, 0)), (1,(8, 0))] 
# -------------------------------------------------
main(f, 'SCS')

1
------------------------
0.999999999999961*x1**12 - 2.17822460654605e-18*x1**11 + 3.56736461482043e-8*x1**10 + 2.1693948281617e-17*x1**9 + 0.999999845146975*x1**8 - 5.6950521433484e-16*x1**7 + 1.79082120632668e-7*x1**6 - 2.72751951436639e-14*x1**5 - 3.53138847209583e-8*x1**4 + 6.26507321710646e-14*x1**3 + 1.59093747450134e-8*x1**2 - 4.75781949972073e-15*x1 + 4.65975228495629e-6


In [260]:
# ---------------- User input ----------------
f = [(1,(12, 0)), (1,(8, 0))] 
# -------------------------------------------------
main(f, 'MOSEK')

1
------------------------
0.999999992370614*x1**12 + 2.59759670072306e-17*x1**11 - 1.99355893638223e-10*x1**10 + 2.74680164291895e-17*x1**9 + 0.999999992391146*x1**8 - 3.39990991506883e-17*x1**7 - 1.45822619913552e-10*x1**6 - 5.03121138468332e-18*x1**5 - 2.20220260701166e-10*x1**4 + 4.94836245647712e-18*x1**3 - 3.70623373638812e-10*x1**2 + 1.35736789074184e-16*x1 + 9.85779444629107e-9


#### **Example 12**
$f(x_1, x_2) = x_1^4-4x_1^3x_2+7x_1^2x_2^2-4x_1x_2^3-4x_1x_2+x_2^4+4$

In [262]:
# ---------------- User input ----------------
f = [(1,(4, 0)), (-4,(3, 1)), (7,(2, 2)), (-4,(1, 3)), (-4,(1, 1)), (1,(0, 4)), (4,(0, 0))] 
# -------------------------------------------------
main(f, 'SCS')

0
------------------------
0.999999885963609*x1**4 - 4.00000000329333*x1**3*x2 + 2.7350291667593e-13*x1**3 + 7.00000009043489*x1**2*x2**2 + 3.0277609529759e-13*x1**2*x2 + 5.75178772557194e-5*x1**2 - 4.0000000032932*x1*x2**3 - 3.01992627021618e-13*x1*x2**2 - 3.99988494828899*x1*x2 - 3.90513451675179e-14*x1 + 0.999999885963708*x2**4 - 2.74499441406181e-13*x2**3 + 5.75178897892492e-5*x2**2 + 3.90259312420825e-14*x2 + 4.00000002642459


In [263]:
# ---------------- User input ----------------
f = [(1,(4, 0)), (-4,(3, 1)), (7,(2, 2)), (-4,(1, 3)), (-4,(1, 1)), (1,(0, 4)), (4,(0, 0))] 
# -------------------------------------------------
main(f, 'MOSEK')

1
------------------------
0.999999999708969*x1**4 - 3.99999999385266*x1**3*x2 - 5.02587471115453e-17*x1**3 + 6.99999999388804*x1**2*x2**2 + 4.43906707543446e-17*x1**2*x2 + 7.35048244493441e-10*x1**2 - 3.99999999385266*x1*x2**3 + 9.64823649187811e-17*x1*x2**2 - 3.99999999458925*x1*x2 - 7.72812150417323e-17*x1 + 0.999999999708969*x2**4 + 5.11746506368305e-16*x2**3 + 7.35048688582651e-10*x2**2 + 1.77654208590194e-16*x2 + 3.99999999554586


#### **Example 13**
$f(x_1, x_2) = x_1^{16}x_2^{16}+x_1^8x_2^8+x_1^8+x_1^4x_2^4+x_1^2x_2^2+x_2^8$

In [264]:
# ---------------- User input ----------------
f = [(1,(8, 0)), (1,(0, 8)), (1,(2, 2)), (1,(4, 4)), (1,(8, 8)), (1,(16, 16))] 
# -------------------------------------------------
main(f, 'SCS')

0
------------------------
2.62614691826901e-5*x1**32 - 8.62346078505159e-12*x1**31*x2 - 1.02910070645528e-11*x1**31 + 4.69094174473682e-5*x1**30*x2**2 + 2.22547547469039e-11*x1**30*x2 + 1.15702432239872e-5*x1**30 + 1.72653356381921e-11*x1**29*x2**3 - 2.22179773805675e-11*x1**29*x2**2 - 1.70128847572759e-10*x1**29*x2 + 6.84237688547637e-11*x1**29 - 8.41461925576666e-5*x1**28*x2**4 - 2.86172050732844e-11*x1**28*x2**3 + 8.69244871457582e-6*x1**28*x2**2 + 3.98338547982232e-10*x1**28*x2 + 7.83549273139686e-6*x1**28 - 1.35745854050533e-11*x1**27*x2**5 + 8.53930650314672e-12*x1**27*x2**4 + 2.15285448300229e-13*x1**27*x2**3 - 2.70305074179114e-10*x1**27*x2**2 + 7.19554246585394e-11*x1**27*x2 + 5.9898014914392e-11*x1**27 - 3.31632626922261e-5*x1**26*x2**6 + 1.45653769380632e-11*x1**26*x2**5 - 1.26582085374181e-5*x1**26*x2**4 + 3.14349228507983e-10*x1**26*x2**3 + 5.59254283846326e-6*x1**26*x2**2 + 2.62110017086196e-10*x1**26*x2 - 1.57130419710526e-5*x1**26 - 9.91509132829729e-12*x1**25*x2**7 + 

In [265]:
# ---------------- User input ----------------
f = [(1,(8, 0)), (1,(0, 8)), (1,(2, 2)), (1,(4, 4)), (1,(8, 8)), (1,(16, 16))] 
# -------------------------------------------------
main(f, 'MOSEK')

0
------------------------
2.13038552992893e-8*x1**32 - 1.71764411482396e-19*x1**31*x2 - 4.84090001779398e-21*x1**31 + 2.05449984092198e-9*x1**30*x2**2 + 1.04703583285738e-24*x1**30*x2 + 1.0076603063715e-8*x1**30 + 1.94254792184813e-19*x1**29*x2**3 - 9.21660997911461e-20*x1**29*x2**2 - 1.58930816321453e-19*x1**29*x2 + 1.35536254174233e-19*x1**29 + 2.32766009571126e-10*x1**28*x2**4 + 5.36260232400962e-19*x1**28*x2**3 + 1.22586924147346e-9*x1**28*x2**2 + 1.59295661883913e-19*x1**28*x2 + 2.98302625019364e-9*x1**28 - 2.07191656249854e-18*x1**27*x2**5 + 3.0262995055983e-19*x1**27*x2**4 + 4.15712856715416e-20*x1**27*x2**3 + 6.0780703247407e-20*x1**27*x2**2 - 3.17101090651043e-19*x1**27*x2 - 2.03327646685657e-19*x1**27 + 2.98959633616337e-11*x1**26*x2**6 - 1.02731627312932e-18*x1**26*x2**5 + 1.50608157286388e-10*x1**26*x2**4 - 2.53752889912227e-18*x1**26*x2**3 + 3.74273269977337e-10*x1**26*x2**2 + 1.50243074115459e-18*x1**26*x2 + 7.96255373285859e-10*x1**26 + 1.22365986703618e-17*x1**25*x2**7

#### **Example 14**
$f(x_1, x_2, x_3) = x_1^2x_3^2+(2x_2+3x_3)^4+x_1^2x_2^4x_3^2$

In [11]:
sp.expand(x1**2*x3**2+(2*x2+3*x3)**4+x1**2*x2**4*x3**2)

x1**2*x2**4*x3**2 + x1**2*x3**2 + 16*x2**4 + 96*x2**3*x3 + 216*x2**2*x3**2 + 216*x2*x3**3 + 81*x3**4

In [13]:
# ---------------- User input ----------------
f = [(1,(2, 4, 2)), (1,(2, 0, 2)), (16,(0, 4, 0)), (96,(0, 3, 1)), (216,(0, 2, 2)), (216,(0, 1, 3)), (81,(0, 0, 4))] 
# -------------------------------------------------
main(f, 'SCS')

0
------------------------
0.000487035534219539*x1**8 + 1.92190635147111e-14*x1**7*x2 + 7.85646830701997e-14*x1**7*x3 + 2.9361817677651e-13*x1**7 - 0.000349731563768828*x1**6*x2**2 + 0.00116810076772126*x1**6*x2*x3 - 6.93798367988635e-12*x1**6*x2 + 0.000245146764036994*x1**6*x3**2 - 1.42474301999153e-11*x1**6*x3 - 0.000143971914450964*x1**6 - 2.71984942217315e-15*x1**5*x2**3 - 2.22832059761499e-13*x1**5*x2**2*x3 - 1.60169595816313e-12*x1**5*x2**2 + 1.47307154704014e-13*x1**5*x2*x3**2 - 1.59310561475321e-12*x1**5*x2*x3 + 6.76101025348963e-13*x1**5*x2 - 1.36977419244891e-13*x1**5*x3**3 + 1.53216176464413e-12*x1**5*x3**2 - 2.40680169628363e-13*x1**5*x3 + 7.12972405967582e-13*x1**5 + 0.000834623657660215*x1**4*x2**4 - 0.000309058145142228*x1**4*x2**3*x3 + 1.94065592384705e-11*x1**4*x2**3 + 0.000437094563727908*x1**4*x2**2*x3**2 + 8.27399859451914e-12*x1**4*x2**2*x3 + 0.000909523754557678*x1**4*x2**2 - 0.00033956758468414*x1**4*x2*x3**3 + 7.91846705395603e-12*x1**4*x2*x3**2 + 0.001253453057

In [15]:
# ---------------- User input ----------------
f = [(1,(2, 4, 2)), (1,(2, 0, 2)), (16,(0, 4, 0)), (96,(0, 3, 1)), (216,(0, 2, 2)), (216,(0, 1, 3)), (81,(0, 0, 4))] 
# -------------------------------------------------
main(f, 'MOSEK')

0
------------------------
2.44940370230141e-10*x1**8 - 1.23146095885079e-22*x1**7*x2 - 2.26258348565496e-21*x1**7*x3 + 4.40510334641226e-21*x1**7 + 1.03462110576233e-10*x1**6*x2**2 + 5.6857108813268e-12*x1**6*x2*x3 - 3.43116152407503e-15*x1**6*x2 + 1.55242541148226e-10*x1**6*x3**2 + 1.47213848421604e-16*x1**6*x3 + 9.18219774481528e-11*x1**6 - 1.52907802266225e-22*x1**5*x2**3 + 1.48430866507517e-18*x1**5*x2**2*x3 - 1.95799967050422e-21*x1**5*x2**2 + 5.15904999407476e-22*x1**5*x2*x3**2 + 3.85579180133637e-20*x1**5*x2*x3 - 8.01801474302979e-22*x1**5*x2 - 2.21210952400977e-21*x1**5*x3**3 + 5.34846654133115e-21*x1**5*x3**2 - 8.69034993629374e-19*x1**5*x3 + 7.05090342622909e-21*x1**5 + 9.62683452646882e-11*x1**4*x2**4 + 1.0324208363086e-11*x1**4*x2**3*x3 - 3.39100717373551e-15*x1**4*x2**3 - 1.59019915878843e-11*x1**4*x2**2*x3**2 + 3.40374181175705e-18*x1**4*x2**2*x3 - 8.21567317073794e-12*x1**4*x2**2 + 1.11699930285861e-11*x1**4*x2*x3**3 - 6.32873683211754e-18*x1**4*x2*x3**2 + 7.68682651710

#### **Example 15**
$f(x_1, x_2, x_3, x_4, x_5, x_6, x_7) = x_1^2x_3^2+(x_1+2x_7x_6-x_2^2)^2+x_4^2+x_5^4$

In [17]:
sp.expand(x1**2*x3**2+(x1+2*x7*x6-x2**2)**2+x4**2*x5**4)

x1**2*x3**2 + x1**2 - 2*x1*x2**2 + 4*x1*x6*x7 + x2**4 - 4*x2**2*x6*x7 + x4**2*x5**4 + 4*x6**2*x7**2

In [18]:
# ---------------- User input ----------------
f = [(1,(2, 0, 2, 0, 0, 0, 0)), (1,(2, 0, 0, 0, 0, 0, 0)), (-2,(1, 2, 0, 0, 0, 0, 0)), (4,(1, 0, 0, 0, 0, 1, 1)), 
    (1,(0, 4, 0, 0, 0, 0, 0)), (-4,(0, 2, 0, 0, 0, 1, 1)), (1,(0, 0, 0, 2, 4, 0, 0)), (4,(0, 0, 0, 0, 0, 2, 2))] 
# -------------------------------------------------
main(f, 'SCS')

0
------------------------
1.42680029571735e-7*x1**6 - 6.39476146932627e-17*x1**5*x2 - 2.83559339320588e-17*x1**5*x3 + 1.81924532934407e-18*x1**5*x4 + 2.12042180667556e-19*x1**5*x5 + 1.64349945759899e-17*x1**5*x6 + 3.4429697232079e-17*x1**5*x7 - 1.11570632905259e-7*x1**5 + 2.44633573461161e-7*x1**4*x2**2 + 3.50826953356441e-16*x1**4*x2*x3 - 7.23466437805645e-18*x1**4*x2*x4 + 2.80518754231407e-17*x1**4*x2*x5 - 1.06424783300472e-15*x1**4*x2*x6 + 9.75039098578082e-16*x1**4*x2*x7 + 6.81779222461888e-17*x1**4*x2 + 2.65935687761477e-7*x1**4*x3**2 + 4.13990557842944e-17*x1**4*x3*x4 + 2.74572864431824e-16*x1**4*x3*x5 - 6.75280812541033e-16*x1**4*x3*x6 + 4.03718869258431e-17*x1**4*x3*x7 - 2.45347788871246e-16*x1**4*x3 + 6.34680343111659e-16*x1**4*x4**2 - 2.96139935824458e-16*x1**4*x4*x5 - 2.20961770189005e-17*x1**4*x4*x6 - 1.05234885433757e-18*x1**4*x4*x7 + 3.44470859135261e-17*x1**4*x4 + 3.16373304325197e-16*x1**4*x5**2 + 8.6455572806294e-17*x1**4*x5*x6 - 7.92066782012552e-18*x1**4*x5*x7 + 2.0

In [19]:
# ---------------- User input ----------------
f = [(1,(2, 0, 2, 0, 0, 0, 0)), (1,(2, 0, 0, 0, 0, 0, 0)), (-2,(1, 2, 0, 0, 0, 0, 0)), (4,(1, 0, 0, 0, 0, 1, 1)), 
    (1,(0, 4, 0, 0, 0, 0, 0)), (-4,(0, 2, 0, 0, 0, 1, 1)), (1,(0, 0, 0, 2, 4, 0, 0)), (4,(0, 0, 0, 0, 0, 2, 2))] 
# -------------------------------------------------
main(f, 'MOSEK')

1
------------------------
3.1847056493858e-10*x1**6 + 2.59680516965564e-18*x1**5*x2 + 3.94273240393879e-18*x1**5*x3 - 9.41471328497237e-17*x1**5*x4 - 3.39212060574741e-18*x1**5*x5 + 3.54006482155712e-17*x1**5*x6 + 4.67748113877354e-17*x1**5*x7 + 2.4803569133978e-11*x1**5 + 3.75985484885644e-10*x1**4*x2**2 - 7.17700236147161e-19*x1**4*x2*x3 + 1.50864953976249e-17*x1**4*x2*x4 - 5.55617360539461e-18*x1**4*x2*x5 - 1.15015969436287e-17*x1**4*x2*x6 - 1.13509874903096e-17*x1**4*x2*x7 - 1.68122590054678e-18*x1**4*x2 + 3.91662641078786e-10*x1**4*x3**2 - 1.17226080447795e-16*x1**4*x3*x4 + 5.68981857838426e-16*x1**4*x3*x5 + 4.87590639974528e-18*x1**4*x3*x6 + 7.82140434143799e-18*x1**4*x3*x7 - 7.27629166669147e-17*x1**4*x3 + 6.70996677895767e-10*x1**4*x4**2 + 3.38304811139718e-17*x1**4*x4*x5 + 9.03358668063886e-17*x1**4*x4*x6 + 3.02145703730249e-16*x1**4*x4*x7 + 1.24492894923076e-16*x1**4*x4 + 2.55261906824983e-10*x1**4*x5**2 - 6.87793844332503e-17*x1**4*x5*x6 - 1.14308122296951e-16*x1**4*x5*x7 -